# How to Use the Web Services API for Subsetting MERRA-2 Data
***

## Overview:
The GES DISC Application Program Interface (API) is intended for users who prefer to employ our data search and subsetting services using scripts instead of the Web browser interface. The API is a communication protocol that allows users to find the datasets and data granules they need and download any desired subsets. Information is passed back and forth in JavaScript Object Notation (JSON) format.

## Example:
The example code provided below demonstrates three examples on how to use the API to submit an asynchronous request to the GES DISC Subsetting Service in order to obtain subsets of the Modern-Era Retropsective analysis for Research and Applications, Version 2 (MERRA-2).

The first example presents a complete workflow that uses the 3-dimensional 3-hourly instantaneous data product M2I3NPASM.

The second example uses the 2-dimensional hourly time-averaged product M2T1NXSLV and highlights the steps which need to be modified when changing the specifics of the subset request.

The third example uses the monthly 2-dimensional time-averaged data product M2TMNXAER rather than subsetting sub-daily data as in the first two examples.

## Prerequisites:
This example code is written in Python3 and requires the following libraries: sys, json, urllib3, certifi, requests, time, http, urllib and getpass. A user must be registered with Earthdata Login and be authorized to access data at the NASA GES DISC. Please read more by visiting [GES DISC data access](https://disc.gsfc.nasa.gov/data-access).

## Updates:
Last updated on Oct 20, 2021

### 1) Example #1: MERRA-2 3-Hourly 3-Dimensional Instantaneous Variables (M2I3NPASM) 

The data product used in this first example is the MERRA-2 product M2I3NPASM which contains instantaneous values at 3-hour intervals of 3-dimensional meteorological fields on pressure levels. Individual data granules for this product are larger than 1 GB and contain data for one day with eight time steps per file. The example shown here will perform subsetting by: variable, spatial domain, vertical dimension, and time of day. This reduces the data volume to 0.3% of it's original size.

The first step is to import the required Python libraries. If any of the following import commands fail check the local Python environment and install any missing packages.

In [1]:
# STEP 1
import sys
import json
import urllib3
import certifi
import requests
from time import sleep
from http.cookiejar import CookieJar
import urllib.request
from urllib.parse import urlencode
import getpass

The second step is to initialize the urllib PoolManager and set the base URL for the API requests that will be sent to the GES DISC subsetting service.

In [2]:
# STEP 2
# Create a urllib PoolManager instance to make requests.
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
# Set the URL for the GES DISC subset service endpoint
url = 'https://disc.gsfc.nasa.gov/service/subset/jsonwsp'

The third step defines a local general-purpose method that submits a JSON-formatted Web Services Protocol (WSP) request to the GES DISC server, checks for any errors, and then returns the response. This method is created for convenience as this task will be repeated more than once.

In [3]:
# STEP 3
# This method POSTs formatted JSON WSP requests to the GES DISC endpoint URL
# It is created for convenience since this task will be repeated more than once
def get_http_data(request):
    hdrs = {'Content-Type': 'application/json',
            'Accept'      : 'application/json'}
    data = json.dumps(request)       
    r = http.request('POST', url, body=data, headers=hdrs)
    response = json.loads(r.data)   
    # Check for errors
    if response['type'] == 'jsonwsp/fault' :
        print('API Error: faulty %s request' % response['methodname'])
        sys.exit(1)
    return response

The fourth step defines the specific details of the subset. In this example three variables are selected: air temperature (T), relative humidity (RH), and ozone mass mixing ratio (O3). The desired spatial coverage is in the Southern Hemisphere poleward of 45S. The date range for our example is 1-5 January 1980 and we will extract data samples only at 00Z. The vertical dimension is restricted to just the mandatory pressure levels. These details are coded as local variables so they can be easily changed for different use cases.

You can use the API to do a [Dataset Search](https://disc.gsfc.nasa.gov/information/howto?keywords=api&title=How%20to%20Use%20the%20Web%20Services%20API%20for%20Dataset%20Searching) in order to find out the exact name of the data product and its variables. Alternatively, a single data granule can be downloaded to find out the variable names. Note that variable names are case sensitive.

In [4]:
# STEP 4
# Define the parameters for the data subset
product = 'M2I3NPASM_V5.12.4' 
varNames =['T', 'RH', 'O3']
minlon = -180
maxlon = 180
minlat = -90
maxlat = -45
begTime = '1980-01-01'
endTime = '1980-01-05'
begHour = '00:00'
endHour = '00:00'

# Subset only the mandatory pressure levels (units are hPa)
# 1000 925 850 700 500 400 300 250 200 150 100 70 50 30 20 10 7 5 3 2 1 
dimName = 'lev'
dimVals = [1,4,7,13,17,19,21,22,23,24,25,26,27,29,30,31,32,33,35,36,37]
# Construct the list of dimension name:value pairs to specify the desired subset
dimSlice = []
for i in range(len(dimVals)):
    dimSlice.append({'dimensionId': dimName, 'dimensionValue': dimVals[i]})

In the fifth step, the desired spatial and temporal constraints, along with the dataset and variable specifications, are stored in a JSON-based Web Service Protocol (WSP) structure, which is named “subset_request”.

In [5]:
# STEP 5
# Construct JSON WSP request for API method: subset
subset_request = {
    'methodname': 'subset',
    'type': 'jsonwsp/request',
    'version': '1.0',
    'args': {
        'role'  : 'subset',
        'start' : begTime,
        'end'   : endTime,
        'box'   : [minlon, minlat, maxlon, maxlat],
        'crop'  : True, 
        'data': [{'datasetId': product,
                  'variable' : varNames[0],
                  'slice': dimSlice
                 },
                 {'datasetId': product,
                  'variable' : varNames[1],
                  'slice'    : dimSlice
                 },
                 {'datasetId': product,
                  'variable' : varNames[2],
                  'slice': dimSlice
                 }]
    }
}

**Note:** The two code blocks above (where the subset parameters are defined and the subset_request structure is assembled) are the only sections of this example that need to be modified for other use cases. Please scroll down to the bottom of this page to see the second example that illustrates how to create a daily average of 2-dimensional hourly variables.

The top level parameters in the subset_request structure shown above are: methodname, type, version, and args. The args contain parameters to customize the subset, which in this case are: role, start, end, box, crop, and data.

For API subset requests the parameters methodname and role should always be set to 'subset'. The remaining parameter values are set using the local variables we defined in the previous code block. The start and end parameters contain the date range. The box parameter specifies the desired spatial domain that will constrain the granule search — only data granules within the spatial domain will be returned. The crop parameter is a True/False flag that indicates whether to perform spatial subsetting on the granules returned by the spatial search. Granules will not be trimmed to the specified spatial domain unless crop is set to True.

The data parameter section includes three sub-parameters: datasetId, variable, and slice. If you wish to retrieve all vertical dimensions, omit the slice parameter; if you wish to retrieve all the variables in the data file, omit the variable parameter. Please refer to the [Complete reference documentation for the GES DISC Subsetting Service API](https://disc.gsfc.nasa.gov/service/subset) for more information.

In the sixth step, the JSON-formatted subset_request is posted to the GES DISC server. The Job ID is extracted from the response and will be used later as a reference for the request.

In [6]:
# STEP 6
# Submit the subset request to the GES DISC Server
response = get_http_data(subset_request)
# Report the JobID and initial status
myJobId = response['result']['jobId']
print('Job ID: '+myJobId)
print('Job status: '+response['result']['Status'])

Job ID: 61706eb29fe5019e4cf03ef3
Job status: Accepted


At this point the job is running on the GES DISC server. The seventh step is to construct another JSON WSP status_request, with methodname parameter set to 'GetStatus'. The args parameter contains the extracted Job ID. The status_request is submitted periodically to monitor the job status as it changes from 'Accepted' to 'Running' to '100% completed'. When the job is finished check on the final status to ensure the job succeeded.

In [7]:
# STEP 7
# Construct JSON WSP request for API method: GetStatus
status_request = {
    'methodname': 'GetStatus',
    'version': '1.0',
    'type': 'jsonwsp/request',
    'args': {'jobId': myJobId}
}

# Check on the job status after a brief nap
while response['result']['Status'] in ['Accepted', 'Running']:
    sleep(5)
    response = get_http_data(status_request)
    status  = response['result']['Status']
    percent = response['result']['PercentCompleted']
    print ('Job status: %s (%d%c complete)' % (status,percent,'%'))
if response['result']['Status'] == 'Succeeded' :
    print ('Job Finished:  %s' % response['result']['message'])
else : 
    print('Job Failed: %s' % response['fault']['code'])
    sys.exit(1)

Job status: Succeeded (100% complete)
Job Finished:  Complete (M2I3NPASM_5.12.4)


After confirming that the job has finished successfully it is time to retrieve the results. The results of a subset request job are URLs: there are HTTP_Services URLs (one for every data granule in the time range of interest) plus links to any relevant documentation. Each HTTP_Services URL contains the specifics of the subset request encoded as facets. Data subsets and documentation files are downloaded using the requests Python library.

There are two ways to retrieve the list of URLs when the subset job is finished:

**Method 1:** Use the API method named GetResult. This method will return the URLs along with three additional attributes: a label, plus the beginning and ending time stamps for that particular data granule. The label serves as the filename for the downloaded subsets.

**Method 2:** Retrieve a plain-text list of URLs in a single shot using the saved JobID. This is a shortcut to retrieve just the list of URLs without any of the other metadata.

The next step for **Method 1** is to construct a third type of JSON WSP request that retrieves the results of this Job. When that request is submitted the results are returned in batches of 20 items, starting with item 0. The startIndex value in the results_request structure must be updated after each successive batch is retrieved.

In [8]:
# STEP 8 (Plan A - preferred)
# Construct JSON WSP request for API method: GetResult
batchsize = 20
results_request = {
    'methodname': 'GetResult',
    'version': '1.0',
    'type': 'jsonwsp/request',
    'args': {
        'jobId': myJobId,
        'count': batchsize,
        'startIndex': 0
    }
}

# Retrieve the results in JSON in multiple batches 
# Initialize variables, then submit the first GetResults request
# Add the results from this batch to the list and increment the count
results = []
count = 0 
response = get_http_data(results_request) 
count = count + response['result']['itemsPerPage']
results.extend(response['result']['items']) 

# Increment the startIndex and keep asking for more results until we have them all
total = response['result']['totalResults']
while count < total :
    results_request['args']['startIndex'] += batchsize 
    response = get_http_data(results_request) 
    count = count + response['result']['itemsPerPage']
    results.extend(response['result']['items'])
       
# Check on the bookkeeping
print('Retrieved %d out of %d expected items' % (len(results), total))

Retrieved 6 out of 6 expected items


Below is the code for **Method 2**. Construct a request using the saved JobID and retrieve the results with the requests library. If the requests.get() method does not return an error, the URLs are stored locally and printed out for informational purposes.

In [ ]:
# STEP 8 (Plan B)
# Retrieve a plain-text list of results in a single shot using the saved JobID

result = requests.get('https://disc.gsfc.nasa.gov/api/jobs/results/'+myJobId)
try:
    result.raise_for_status()
    urls = result.text.split('\n')
    for i in urls : print('\n%s' % i)
except :
    print('Request returned error code %d' % result.status_code)


It is important to keep in mind that the results returned at this point are not data files but lists of URLs. Most of the URLs will contain HTTP_Services requests to actually do the subsetting and return the data, but some of them may be links to documentation files pertaining to the dataset in question.

It is worthwhile to separate the document URLs from the HTTP_services URLs in case the documentation has already been retrieved so they won't be downloaded again. The way we do this is to check for start and end attributes which are always associated with HTTP_services URLs. The remainder of the example code assumes the use of **Method 1** because it makes use of this extra metadata.

In [9]:
# Sort the results into documents and URLs

docs = []
urls = []
for item in results :
    try:
        if item['start'] and item['end'] : urls.append(item) 
    except:
        docs.append(item)
# Print out the documentation links, but do not download them
# print('\nDocumentation:')
# for item in docs : print(item['label']+': '+item['link'])

The final step is to invoke each HTTP_Services URL and download the data files. The contents of the label attribute are used here as the output file name, but the name can be any string. It is important to download each file one at a time, in series rather than in parallel, to avoid overloading the GES DISC servers.

We show two methods for downloading the data files using either the requests.get() or the urllib.request() modules. Use one or the other, but not both. If the requests.get() method fails try the alternate code block, but be sure to update it with a proper Earthdata login name and password.

**Download with Requests Library:**  
In STEP 10, for the request.get() module to work properly, you must have a [HOME/.netrc](https://disc.gsfc.nasa.gov/data-access) file that contains the following text (configured with your own Earthdata userid and password): machine urs.earthdata.nasa.gov login [userid] password [password]. In the alternate STEP 10, you can provide your userid and password on-the-fly when running the code.

In [11]:
# STEP 10 
# Use the requests library to submit the HTTP_Services URLs and write out the results.
print('\nHTTP_services output:')
for item in urls :
    URL = item['link'] 
    result = requests.get(URL)
    try:
        result.raise_for_status()
        outfn = item['label']
        f = open(outfn,'wb')
        f.write(result.content)
        f.close()
        print(outfn, "is downloaded")
    except:
        print('Error! Status code is %d for this URL:\n%s' % (result.status.code,URL))
        print('Help for downloading data is at https://disc.gsfc.nasa.gov/data-access')
        
print('Downloading is done and find the downloaded files in your current working directory')


HTTP_services output:
MERRA2_100.inst3_3d_asm_Np.19800101.SUB.nc is downloaded
MERRA2_100.inst3_3d_asm_Np.19800102.SUB.nc is downloaded
MERRA2_100.inst3_3d_asm_Np.19800103.SUB.nc is downloaded
MERRA2_100.inst3_3d_asm_Np.19800104.SUB.nc is downloaded
MERRA2_100.inst3_3d_asm_Np.19800105.SUB.nc is downloaded


**Alternative Download Method Using Native Python:**  
If the code above does not work in your local environment try this alternate method. Please enter your Earthdata userid and password when prompted while running the code.

In [ ]:
# ATLERNATIVE STEP 10 
# Create a password manager to deal with the 401 response that is returned from
# Earthdata Login

# Create a password manager to deal with the 401 response that is returned from
# Earthdata Login

username = input("Provide your EarthData userid: ")
password = getpass.getpass("Provide your EarthData password: ")

password_manager = urllib.request.HTTPPasswordMgrWithDefaultRealm()
password_manager.add_password(None, "https://urs.earthdata.nasa.gov", username, password)

# Create a cookie jar for storing cookies. This is used to store and return the session cookie #given to use by the data server
cookie_jar = CookieJar()
   
# Install all the handlers.
opener = urllib.request.build_opener (urllib.request.HTTPBasicAuthHandler (password_manager),urllib.request.HTTPCookieProcessor (cookie_jar))
urllib.request.install_opener(opener)
 
# Open a request for the data, and download files
print('\nHTTP_services output:')
for item in urls:
    URL = item['link'] 
    DataRequest = urllib.request.Request(URL)
    DataResponse = urllib.request.urlopen(DataRequest)

# Print out the result
    DataBody = DataResponse.read()

# Save file to working directory
    try:
        file_name = item['label']
        file_ = open(file_name, 'wb')
        file_.write(DataBody)
        file_.close()
        print (file_name, "is downloaded")
    except requests.exceptions.HTTPError as e:
         print(e)
            
print('Downloading is done and find the downloaded files in your current working directory')

### 2) Example #2: MERRA-2 Hourly 2-Dimensional Time-Averaged Variables (M2T1NXSLV) 

The second example uses the MERRA-2 product M2T1NXSLV, which contains time-averaged values of 2-dimensional meteorological fields at hourly intervals. For this case, we will create a global daily mean by averaging all the time steps in each granule, and also regrid the result. All the steps in the workflow are the same as those outlined above, except for Steps 4 and 5, which are rewritten below. For this example, the details of the defined parameters that describe the subset are different, and the JSON-formatted subset_request structure is altered to disable the vertical dimension slicing and enable the regridding feature.

The selected variables in this example are: total precipitable water vapor (TQV), total precipitable liquid water (TQL), and total precipitable ice water(TQI). The spatial coverage is global, and the date range remains 4-5 January 1980. Because we are working with 2-dimensional data, there is no dimensional subsetting so the dimSlice parameter is omitted.

Because these hourly data variables are time-averaged instead of instantaneous, the time stamp is in the middle of the hour (00:30Z) instead of at the top of the hour (00:00Z) as in the previous example. Note that the result will have a time stamp of 12:00Z, the mid point of the averaging period. The options for diurnalAggregation are: '1' (daily mean), '2' (daily minimum), and '3' (daily maximum).

We will also enable the regridding feature by specifying an interpolation method and a destination grid. This regridding service is not available for all GES DISC data products, but it is an option for Level 3 or Level 4 data collections. The options for interpolation methods are: 'remapbil', 'remapbic', 'remapdis', 'remapnn', 'remapcon', 'remapcon2', 'remaplaf'. The options for destination grid are: 'JRA-55', '20cr2x2', 'MERRA1.25', 'gpcp2.5', 'cfsr0.5a', 'cfsr0.5b', 'cfsr1.0', 'cfsr2.5', 'ncepncar2.5', 'geos1x125', 'geos1x1', 'geos4x5', 'geos2x25', 'fv1x125', 'fv2x25', 'fv4x5', 'ERA-40', 'ERA2.5', 'ERA-I', 'ERA1.5', 'ERA.75', 'GPCC2.5', 'GPCC1.0', 'GPCC0.5', 'CMORPH2.5', 'GLDAS-2_1', 'GFDL'. The detailed information on Level 3 and L4 regridder and subsetter can be found [here](https://disc.gsfc.nasa.gov/information/documents?keywords=grid&title=Level%203%20and%204%20Regridder%20and%20Subsetter%20Information).

Note: The options to control regridding are case sensitive. An incorrect spelling or syntax of the interpolation method or destination grid name will result in an error.  

In [ ]:
# STEP 4
# Define the parameters for the second subset example
product = 'M2T1NXSLV_5.12.4'
varNames =['TQV', 'TQL', 'TQI']
minlon = -180
maxlon = 180
minlat = -90
maxlat = 90
begTime = '1980-01-04'
endTime = '1980-01-05'

diurnalAggregation = '1'
interp = 'remapbil'
destGrid = 'cfsr0.5a'

The desired spatial and temporal constraints, along with the dataset and variable specifications, are stored in a JSON-based Web Service Protocol (WSP) structure, which is named “subset_request”. Two additional parameters were added to enable the regridding feature: <code>mapping</code> and <code>grid</code>. 

In [ ]:
# STEP 5
# Construct JSON WSP request for API method: subset
subset_request = {
    'methodname': 'subset',
    'type': 'jsonwsp/request',
    'version': '1.0',
    'args': {
        'role'  : 'subset',
        'start' : begTime,
        'end'   : endTime,
        'box'   : [minlon, minlat, maxlon, maxlat],
        'crop'  : True,
        'diurnalAggregation': diurnalAggregation,
        'mapping': interp,
        'grid'  : destGrid,
        'data': [{'datasetId': product,
                  'variable' : varNames[0]
                 },
                 {'datasetId': product,
                  'variable' : varNames[1]
                 },
                 {'datasetId': product,
                  'variable' : varNames[2]
                 }]
    }
}

The two code blocks above replace Steps 4 and 5 in the complete workflow outlined in Example #1.

### 3) Example #3: MERRA-2 Monthly 2-Dimensional Time-Averaged Variables (M2TMNXAER)**

The third example uses the MERRA-2 product M2TMNXAER, which contains a time-averaged 2-dimensional monthly mean data collection. For this case, we will crop the bounding box with the latitude [-90,0] and longitude [-180,0]. All the steps in the workflow are the same as those outlined above, except for Steps 4 and 5, which are rewritten below. For this example, the details of the defined parameters that describe the subset are different, and the JSON-formatted subset_request structure is altered to disable the vertical dimension slicing and enable the regridding feature.

The selected variable in this example is: so4 extinction [550 nm] (SUEXTTAU). The spatial coverage is a quarter of the globe, and the date range is January and February 1980.

In [ ]:
# STEP 4
# Define the parameters for the third subset example
product = 'M2TMNXAER_5.12.4'
varNames =['SUEXTTAU']
minlon = -180
maxlon = 0
minlat = -90
maxlat = 0
begTime = '1980-01'
endTime = '1980-02'
interp = 'remapbil'
destGrid = 'fv4x5'

The desired spatial and temporal constraints, along with the dataset and variable specifications, are stored in a JSON-based Web Service Protocol (WSP) structure, which is named “subset_request”. Two additional parameters were added to enable the regridding feature: mapping and grid. Comment out them if you don't need regridding.

In [ ]:
# STEP 5
# Construct JSON WSP request for API method: subset
subset_request = {
    'methodname': 'subset',
    'type': 'jsonwsp/request',
    'version': '1.0',
    'args': {
        'role'  : 'subset',
        'start' : begTime,
        'end'   : endTime,
        'box'   : [minlon, minlat, maxlon, maxlat],
        'crop'  : True, 
        'mapping': interp,
        'grid'  : destGrid,
        'data': [{'datasetId': product,
                  'variable' : varNames[0]
                 }]
    }
}

The two code blocks above replace Steps 4 and 5 in the complete workflow outlined in Example #1.

**Additional Info:**  
[How to Use the Web Services API for Subsetting](https://disc.gsfc.nasa.gov/information/howto?keywords=api&title=How%20to%20Use%20the%20Web%20Services%20API%20for%20Subsetting)  
[How to Use the Web Services API for Dataset Searching](https://disc.gsfc.nasa.gov/information/howto?keywords=api&title=How%20to%20Use%20the%20Web%20Services%20API%20for%20Dataset%20Searching)  
[Complete reference documentation for the GES DISC Subsetting Service API](https://disc.gsfc.nasa.gov/service/subset) 

<font size="1">THE SUBJECT FILE IS PROVIDED "AS IS" WITHOUT ANY WARRANTY OF ANY KIND, EITHER EXPRESSED, IMPLIED, OR STATUTORY, INCLUDING, BUT NOT LIMITED TO, ANY WARRANTY THAT THE SUBJECT FILE WILL CONFORM TO SPECIFICATIONS, ANY IMPLIED WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE, OR FREEDOM FROM INFRINGEMENT, ANY WARRANTY THAT THE SUBJECT FILE WILL BE ERROR FREE, OR ANY WARRANTY THAT DOCUMENTATION, IF PROVIDED, WILL CONFORM TO THE SUBJECT FILE. THIS AGREEMENT DOES NOT, IN ANY MANNER, CONSTITUTE AN ENDORSEMENT BY GOVERNMENT AGENCY OR ANY PRIOR RECIPIENT OF ANY RESULTS, RESULTING DESIGNS, HARDWARE, SOFTWARE PRODUCTS OR ANY OTHER APPLICATIONS RESULTING FROM USE OF THE SUBJECT FILE. FURTHER, GOVERNMENT AGENCY DISCLAIMS ALL WARRANTIES AND LIABILITIES REGARDING THIRD-PARTY SOFTWARE, IF PRESENT IN THE SUBJECT FILE, AND DISTRIBUTES IT "AS IS."